In [1]:
import os
import csv
import time
import datetime
import random
import json

import warnings
from collections import Counter
from math import sqrt

import gensim
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
warnings.filterwarnings("ignore")

In [2]:
# 配置参数

class TrainingConfig(object):
    epoches = 4
    evaluateEvery = 100
    checkpointEvery = 100
    learningRate = 0.001
    
class ModelConfig(object):
    embeddingSize = 200
    
    hiddenSizes = [256, 128]  # LSTM结构的神经元个数
    
    dropoutKeepProb = 0.5
    l2RegLambda = 0.0
    
class Config(object):
    sequenceLength = 200  # 取了所有序列长度的均值
    batchSize = 128
    
    dataSource = "../data/preProcess/labeledTrain.csv"
    
    stopWordSource = "../data/english"
    
    numClasses = 2
    
    rate = 0.8  # 训练集的比例
    
    training = TrainingConfig()
    
    model = ModelConfig()

    
# 实例化配置参数对象
config = Config()

In [3]:
# 数据预处理的类，生成训练集和测试集

class Dataset(object):
    def __init__(self, config):
        self._dataSource = config.dataSource
        self._stopWordSource = config.stopWordSource  
        
        self._sequenceLength = config.sequenceLength  # 每条输入的序列处理为定长
        self._embeddingSize = config.model.embeddingSize
        self._batchSize = config.batchSize
        self._rate = config.rate
        
        self._stopWordDict = {}
        
        self.trainReviews = []
        self.trainLabels = []
        
        self.evalReviews = []
        self.evalLabels = []
        
        self.wordEmbedding =None
        
        self._wordToIndex = {}
        self._indexToWord = {}
        
    def _readData(self, filePath):
        """
        从csv文件中读取数据集
        """
        
        df = pd.read_csv(filePath)
        labels = df["sentiment"].tolist()
        review = df["review"].tolist()
        reviews = [line.strip().split() for line in review]

        return reviews, labels

    def _reviewProcess(self, review, sequenceLength, wordToIndex):
        """
        将数据集中的每条评论用index表示
        wordToIndex中“pad”对应的index为0
        """
        
        reviewVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        
        # 判断当前的序列是否小于定义的固定序列长度
        if len(review) < sequenceLength:
            sequenceLen = len(review)
            
        for i in range(sequenceLen):
            if review[i] in wordToIndex:
                reviewVec[i] = wordToIndex[review[i]]
            else:
                reviewVec[i] = wordToIndex["UNK"]

        return reviewVec

    def _genTrainEvalData(self, x, y, rate):
        """
        生成训练集和验证集
        """
        
        reviews = []
        labels = []
        
        # 遍历所有的文本，将文本中的词转换成index表示
        for i in range(len(x)):
            reviewVec = self._reviewProcess(x[i], self._sequenceLength, self._wordToIndex)
            reviews.append(reviewVec)
            
            labels.append([y[i]])
            
        trainIndex = int(len(x) * rate)
        
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(labels[:trainIndex], dtype="float32")
        
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(labels[trainIndex:], dtype="float32")

        return trainReviews, trainLabels, evalReviews, evalLabels
        
    def _genVocabulary(self, reviews):
        """
        生成词向量和词汇-索引映射字典，可以用全数据集
        """
        
        allWords = [word for review in reviews for word in review]
        
        # 去掉停用词
        subWords = [word for word in allWords if word not in self.stopWordDict]
        
        wordCount = Counter(subWords)  # 统计词频
        sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        
        # 去除低频词
        words = [item[0] for item in sortWordCount if item[1] >= 5]
        
        vocab, wordEmbedding = self._getWordEmbedding(words)
        self.wordEmbedding = wordEmbedding
        
        self._wordToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self._indexToWord = dict(zip(list(range(len(vocab))), vocab))
        
        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open("../data/wordJson/wordToIndex.json", "w", encoding="utf-8") as f:
            json.dump(self._wordToIndex, f)
        
        with open("../data/wordJson/indexToWord.json", "w", encoding="utf-8") as f:
            json.dump(self._indexToWord, f)
            
    def _getWordEmbedding(self, words):
        """
        按照我们的数据集中的单词取出预训练好的word2vec中的词向量
        """
        
        wordVec = gensim.models.KeyedVectors.load_word2vec_format("../word2vec/word2Vec.bin", binary=True)
        vocab = []
        wordEmbedding = []
        
        # 添加 "pad" 和 "UNK", 
        vocab.append("pad")
        vocab.append("UNK")
        wordEmbedding.append(np.zeros(self._embeddingSize))
        wordEmbedding.append(np.zeros(self._embeddingSize))
        
        for word in words:
            try:
                vector = wordVec.wv[word]
                vocab.append(word)
                wordEmbedding.append(vector)
            except:
                print(word + "不存在于词向量中")
                
        return vocab, np.array(wordEmbedding)
    
    def _readStopWord(self, stopWordPath):
        """
        读取停用词
        """
        
        with open(stopWordPath, "r") as f:
            stopWords = f.read()
            stopWordList = stopWords.splitlines()
            # 将停用词用列表的形式生成，之后查找停用词时会比较快
            self.stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
            
    def dataGen(self):
        """
        初始化训练集和验证集
        """
        
        # 初始化停用词
        self._readStopWord(self._stopWordSource)
        
        # 初始化数据集
        reviews, labels = self._readData(self._dataSource)
        
        # 初始化词汇-索引映射表和词向量矩阵
        self._genVocabulary(reviews)
        
        # 初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self._genTrainEvalData(reviews, labels, self._rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels
        
        
data = Dataset(config)
data.dataGen()

In [4]:
print("train data shape: {}".format(data.trainReviews.shape))
print("train label shape: {}".format(data.trainLabels.shape))
print("eval data shape: {}".format(data.evalReviews.shape))

train data shape: (20000, 200)
train label shape: (20000, 1)
eval data shape: (5000, 200)


In [5]:
# 输出batch数据集

def nextBatch(x, y, batchSize):
        """
        生成batch数据集，用生成器的方式输出
        """
    
        perm = np.arange(len(x))
        np.random.shuffle(perm)
        x = x[perm]
        y = y[perm]
        
        numBatches = len(x) // batchSize

        for i in range(numBatches):
            start = i * batchSize
            end = start + batchSize
            batchX = np.array(x[start: end], dtype="int64")
            batchY = np.array(y[start: end], dtype="float32")
            
            yield batchX, batchY

In [6]:
# 构建模型
class BiLSTMAttention(object):
    """
    Text CNN 用于文本分类
    """
    def __init__(self, config, wordEmbedding):

        # 定义模型的输入
        self.inputX = tf.placeholder(tf.int32, [None, config.sequenceLength], name="inputX")
        self.inputY = tf.placeholder(tf.float32, [None, 1], name="inputY")
        
        self.dropoutKeepProb = tf.placeholder(tf.float32, name="dropoutKeepProb")
        
        # 定义l2损失
        l2Loss = tf.constant(0.0)
        
        # 词嵌入层
        with tf.name_scope("embedding"):

            # 利用预训练的词向量初始化词嵌入矩阵
            self.W = tf.Variable(tf.cast(wordEmbedding, dtype=tf.float32, name="word2vec") ,name="W")
            # 利用词嵌入矩阵将输入的数据中的词转换成词向量，维度[batch_size, sequence_length, embedding_size]
            self.embeddedWords = tf.nn.embedding_lookup(self.W, self.inputX)
            
        # 定义两层双向LSTM的模型结构
        with tf.name_scope("Bi-LSTM"):
            for idx, hiddenSize in enumerate(config.model.hiddenSizes):
                with tf.name_scope("Bi-LSTM" + str(idx)):
                    # 定义前向LSTM结构
                    lstmFwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                                                                 output_keep_prob=self.dropoutKeepProb)
                    # 定义反向LSTM结构
                    lstmBwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                                                                 output_keep_prob=self.dropoutKeepProb)


                    # 采用动态rnn，可以动态的输入序列的长度，若没有输入，则取序列的全长
                    # outputs是一个元祖(output_fw, output_bw)，其中两个元素的维度都是[batch_size, max_time, hidden_size],fw和bw的hidden_size一样
                    # self.current_state 是最终的状态，二元组(state_fw, state_bw)，state_fw=[batch_size, s]，s是一个元祖(h, c)
                    outputs_, self.current_state = tf.nn.bidirectional_dynamic_rnn(lstmFwCell, lstmBwCell, 
                                                                                  self.embeddedWords, dtype=tf.float32,
                                                                                  scope="bi-lstm" + str(idx))
        
                    # 对outputs中的fw和bw的结果拼接 [batch_size, time_step, hidden_size * 2], 传入到下一层Bi-LSTM中
                    self.embeddedWords = tf.concat(outputs_, 2)
                
        # 将最后一层Bi-LSTM输出的结果分割成前向和后向的输出
        outputs = tf.split(self.embeddedWords, 2, -1)
        
        # 在Bi-LSTM+Attention的论文中，将前向和后向的输出相加
        with tf.name_scope("Attention"):
            H = outputs[0] + outputs[1]

            # 得到Attention的输出
            output = self.attention(H)
            outputSize = config.model.hiddenSizes[-1]
        
        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW",
                shape=[outputSize, 1],
                initializer=tf.contrib.layers.xavier_initializer())
            
            outputB= tf.Variable(tf.constant(0.1, shape=[1]), name="outputB")
            l2Loss += tf.nn.l2_loss(outputW)
            l2Loss += tf.nn.l2_loss(outputB)
            self.predictions = tf.nn.xw_plus_b(output, outputW, outputB, name="predictions")
            self.binaryPreds = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
        
        # 计算二元交叉熵损失
        with tf.name_scope("loss"):
            
            losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
            self.loss = tf.reduce_mean(losses) + config.model.l2RegLambda * l2Loss
    
    def attention(self, H):
        """
        利用Attention机制得到句子的向量表示
        """
        # 获得最后一层LSTM的神经元数量
        hiddenSize = config.model.hiddenSizes[-1]
        
        # 初始化一个权重向量，是可训练的参数
        W = tf.Variable(tf.random_normal([hiddenSize], stddev=0.1))
        
        # 对Bi-LSTM的输出用激活函数做非线性转换
        M = tf.tanh(H)
        
        # 对W和M做矩阵运算，W=[batch_size, time_step, hidden_size]，计算前做维度转换成[batch_size * time_step, hidden_size]
        # newM = [batch_size, time_step, 1]，每一个时间步的输出由向量转换成一个数字
        newM = tf.matmul(tf.reshape(M, [-1, hiddenSize]), tf.reshape(W, [-1, 1]))
        
        # 对newM做维度转换成[batch_size, time_step]
        restoreM = tf.reshape(newM, [-1, config.sequenceLength])
        
        # 用softmax做归一化处理[batch_size, time_step]
        self.alpha = tf.nn.softmax(restoreM)
        
        # 利用求得的alpha的值对H进行加权求和，用矩阵运算直接操作
        r = tf.matmul(tf.transpose(H, [0, 2, 1]), tf.reshape(self.alpha, [-1, config.sequenceLength, 1]))
        
        # 将三维压缩成二维sequeezeR=[batch_size, hidden_size]
        sequeezeR = tf.squeeze(r)
        
        sentenceRepren = tf.tanh(sequeezeR)
        
        # 对Attention的输出可以做dropout处理
        output = tf.nn.dropout(sentenceRepren, self.dropoutKeepProb)
        
        return output

In [7]:
# 定义性能指标函数

def mean(item):
    return sum(item) / len(item)


def genMetrics(trueY, predY, binaryPredY):
    """
    生成acc和auc值
    """
    auc = roc_auc_score(trueY, predY)
    accuracy = accuracy_score(trueY, binaryPredY)
    precision = precision_score(trueY, binaryPredY)
    recall = recall_score(trueY, binaryPredY)
    
    return round(accuracy, 4), round(auc, 4), round(precision, 4), round(recall, 4)

In [ ]:
# 训练模型

# 生成训练集和验证集
trainReviews = data.trainReviews
trainLabels = data.trainLabels
evalReviews = data.evalReviews
evalLabels = data.evalLabels

wordEmbedding = data.wordEmbedding

# 定义计算图
with tf.Graph().as_default():

    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth=True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9  # 配置gpu占用率  

    sess = tf.Session(config=session_conf)
    
    # 定义会话
    with sess.as_default():
        lstm = BiLSTMAttention(config, wordEmbedding)
        
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(config.training.learningRate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(lstm.loss)
        # 将梯度应用到变量下，生成训练器
        trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)
        
        # 用summary绘制tensorBoard
        gradSummaries = []
        for g, v in gradsAndVars:
            if g is not None:
                tf.summary.histogram("{}/grad/hist".format(v.name), g)
                tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        
        outDir = os.path.abspath(os.path.join(os.path.curdir, "summarys"))
        print("Writing to {}\n".format(outDir))
        
        lossSummary = tf.summary.scalar("loss", lstm.loss)
        summaryOp = tf.summary.merge_all()
        
        trainSummaryDir = os.path.join(outDir, "train")
        trainSummaryWriter = tf.summary.FileWriter(trainSummaryDir, sess.graph)
        
        evalSummaryDir = os.path.join(outDir, "eval")
        evalSummaryWriter = tf.summary.FileWriter(evalSummaryDir, sess.graph)
        
        
        # 初始化所有变量
#         saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
        
        # 保存模型的一种方式，保存为pb文件
#         builder = tf.saved_model.builder.SavedModelBuilder("../model/Bi-LSTM/savedModel")
        sess.run(tf.global_variables_initializer())

        def trainStep(batchX, batchY):
            """
            训练函数
            """   
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: config.model.dropoutKeepProb
            }
            _, summary, step, loss, predictions, binaryPreds = sess.run(
                [trainOp, summaryOp, globalStep, lstm.loss, lstm.predictions, lstm.binaryPreds],
                feed_dict)
            timeStr = datetime.datetime.now().isoformat()
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(timeStr, step, loss, acc, auc, precision, recall))
            trainSummaryWriter.add_summary(summary, step)

        def devStep(batchX, batchY):
            """
            验证函数
            """
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: 1.0
            }
            summary, step, loss, predictions, binaryPreds = sess.run(
                [summaryOp, globalStep, lstm.loss, lstm.predictions, lstm.binaryPreds],
                feed_dict)
            
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            
            evalSummaryWriter.add_summary(summary, step)
            
            return loss, acc, auc, precision, recall
        
        for i in range(config.training.epoches):
            # 训练模型
            print("start training model")
            for batchTrain in nextBatch(trainReviews, trainLabels, config.batchSize):
                trainStep(batchTrain[0], batchTrain[1])

                currentStep = tf.train.global_step(sess, globalStep) 
                if currentStep % config.training.evaluateEvery == 0:
                    print("\nEvaluation:")
                    
                    losses = []
                    accs = []
                    aucs = []
                    precisions = []
                    recalls = []
                    
                    for batchEval in nextBatch(evalReviews, evalLabels, config.batchSize):
                        loss, acc, auc, precision, recall = devStep(batchEval[0], batchEval[1])
                        losses.append(loss)
                        accs.append(acc)
                        aucs.append(auc)
                        precisions.append(precision)
                        recalls.append(recall)
                        
                    time_str = datetime.datetime.now().isoformat()
                    print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(time_str, currentStep, mean(losses), 
                                                                                                       mean(accs), mean(aucs), mean(precisions),
                                                                                                       mean(recalls)))
                    
#                 if currentStep % config.training.checkpointEvery == 0:
#                     # 保存模型的另一种方法，保存checkpoint文件
#                     path = saver.save(sess, "../model/Bi-LSTM/model/my-model", global_step=currentStep)
#                     print("Saved model checkpoint to {}\n".format(path))
                    
#         inputs = {"inputX": tf.saved_model.utils.build_tensor_info(lstm.inputX),
#                   "keepProb": tf.saved_model.utils.build_tensor_info(lstm.dropoutKeepProb)}

#         outputs = {"binaryPreds": tf.saved_model.utils.build_tensor_info(lstm.binaryPreds)}

#         prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(inputs=inputs, outputs=outputs,
#                                                                                       method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
#         legacy_init_op = tf.group(tf.tables_initializer(), name="legacy_init_op")
#         builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],
#                                             signature_def_map={"predict": prediction_signature}, legacy_init_op=legacy_init_op)

#         builder.save()

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/kernel:0/grad/hist is illegal; using bi-lstm0/fw/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/kernel:0/grad/sparsity is illegal; using bi-lstm0/fw/lstm_cell/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/bias:0/grad/hist is illegal; using bi-lstm0/fw/lstm_cell/bias_0/grad/hist instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/bias:0/grad/sparsity is illegal; using bi-lstm0/fw/lstm_cell/bias_0/grad/sparsity instead.
INFO:tensorflow:Summary name bi-lstm0/bw/lstm_cell/kernel:0/grad/hist is illegal; using bi-lstm0/bw/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name bi-lstm0/bw/lstm_cell/kernel:0/grad/sparsity is illegal; 

2019-01-17T20:59:34.169483, step: 42, loss: 0.30529019236564636, acc: 0.875, auc: 0.9502, precision: 0.9153, recall: 0.8308
2019-01-17T20:59:34.935471, step: 43, loss: 0.36477425694465637, acc: 0.8438, auc: 0.9325, precision: 0.8592, recall: 0.8592
2019-01-17T20:59:35.616078, step: 44, loss: 0.3923361301422119, acc: 0.8672, auc: 0.9247, precision: 0.8462, recall: 0.8871
2019-01-17T20:59:36.354007, step: 45, loss: 0.3005450367927551, acc: 0.8828, auc: 0.9435, precision: 0.8923, recall: 0.8788
2019-01-17T20:59:37.065378, step: 46, loss: 0.39613187313079834, acc: 0.7891, auc: 0.9091, precision: 0.8833, recall: 0.726
2019-01-17T20:59:37.811058, step: 47, loss: 0.28269505500793457, acc: 0.8672, auc: 0.9633, precision: 0.9623, recall: 0.7727
2019-01-17T20:59:38.540161, step: 48, loss: 0.35594069957733154, acc: 0.875, auc: 0.9243, precision: 0.9, recall: 0.8438
2019-01-17T20:59:39.220601, step: 49, loss: 0.2602846622467041, acc: 0.9219, auc: 0.9653, precision: 0.9429, recall: 0.9167
2019-01-1

2019-01-17T21:00:48.047387, step: 107, loss: 0.38088536262512207, acc: 0.8047, auc: 0.9386, precision: 0.9583, recall: 0.6667
2019-01-17T21:00:48.820397, step: 108, loss: 0.30188071727752686, acc: 0.8359, auc: 0.9485, precision: 0.9111, recall: 0.7069
2019-01-17T21:00:49.539057, step: 109, loss: 0.2493475079536438, acc: 0.8828, auc: 0.968, precision: 0.9796, recall: 0.7742
2019-01-17T21:00:50.295764, step: 110, loss: 0.24506689608097076, acc: 0.9219, auc: 0.9761, precision: 0.9167, recall: 0.9429
2019-01-17T21:00:51.018270, step: 111, loss: 0.39011698961257935, acc: 0.8359, auc: 0.9266, precision: 0.7903, recall: 0.8596
2019-01-17T21:00:51.707186, step: 112, loss: 0.28663158416748047, acc: 0.875, auc: 0.9539, precision: 0.9524, recall: 0.8219
2019-01-17T21:00:52.386520, step: 113, loss: 0.3912290930747986, acc: 0.8516, auc: 0.9132, precision: 0.8615, recall: 0.8485
2019-01-17T21:00:53.101394, step: 114, loss: 0.3116287589073181, acc: 0.8516, auc: 0.9458, precision: 0.9583, recall: 0.73

2019-01-17T21:01:34.662296, step: 173, loss: 0.20858250558376312, acc: 0.9141, auc: 0.9749, precision: 0.9649, recall: 0.8594
2019-01-17T21:01:35.382455, step: 174, loss: 0.19043567776679993, acc: 0.9062, auc: 0.9873, precision: 1.0, recall: 0.8095
2019-01-17T21:01:36.100366, step: 175, loss: 0.2959415316581726, acc: 0.8516, auc: 0.9608, precision: 1.0, recall: 0.6833
2019-01-17T21:01:36.741669, step: 176, loss: 0.19541166722774506, acc: 0.8828, auc: 0.9764, precision: 0.9388, recall: 0.7931
2019-01-17T21:01:37.445211, step: 177, loss: 0.23847684264183044, acc: 0.8984, auc: 0.9655, precision: 0.9, recall: 0.8852
2019-01-17T21:01:38.148204, step: 178, loss: 0.1872313767671585, acc: 0.9375, auc: 0.9789, precision: 0.9552, recall: 0.9275
2019-01-17T21:01:38.840816, step: 179, loss: 0.23561348021030426, acc: 0.9297, auc: 0.9609, precision: 0.9516, recall: 0.9077
2019-01-17T21:01:39.584133, step: 180, loss: 0.19130921363830566, acc: 0.9062, auc: 0.9862, precision: 0.8919, recall: 0.9429
201

2019-01-17T21:02:47.643310, step: 238, loss: 0.21910765767097473, acc: 0.9297, auc: 0.9904, precision: 0.8806, recall: 0.9833
2019-01-17T21:02:48.310941, step: 239, loss: 0.22293990850448608, acc: 0.9062, auc: 0.9751, precision: 0.8939, recall: 0.9219
2019-01-17T21:02:49.010640, step: 240, loss: 0.2462482452392578, acc: 0.9062, auc: 0.9648, precision: 0.9032, recall: 0.9032
2019-01-17T21:02:49.704750, step: 241, loss: 0.20884238183498383, acc: 0.9375, auc: 0.9711, precision: 0.9796, recall: 0.8727
2019-01-17T21:02:50.413171, step: 242, loss: 0.20496313273906708, acc: 0.8906, auc: 0.9822, precision: 0.963, recall: 0.8125
2019-01-17T21:02:51.104267, step: 243, loss: 0.21850597858428955, acc: 0.8672, auc: 0.9765, precision: 0.9412, recall: 0.7742
2019-01-17T21:02:51.824423, step: 244, loss: 0.19041579961776733, acc: 0.9141, auc: 0.9809, precision: 0.9552, recall: 0.8889
2019-01-17T21:02:52.539412, step: 245, loss: 0.2131832242012024, acc: 0.9219, auc: 0.9754, precision: 0.9138, recall: 0.

2019-01-17T21:04:00.828065, step: 302, loss: 0.20460377633571625, acc: 0.8984, auc: 0.979, precision: 0.9821, recall: 0.8209
2019-01-17T21:04:01.512212, step: 303, loss: 0.2109317183494568, acc: 0.9062, auc: 0.9761, precision: 0.9194, recall: 0.8906
2019-01-17T21:04:02.237354, step: 304, loss: 0.30218616127967834, acc: 0.8672, auc: 0.9477, precision: 0.9138, recall: 0.8154
2019-01-17T21:04:02.955572, step: 305, loss: 0.19454339146614075, acc: 0.9219, auc: 0.9835, precision: 0.9836, recall: 0.8696
2019-01-17T21:04:03.714798, step: 306, loss: 0.15619586408138275, acc: 0.9688, auc: 0.9828, precision: 0.9821, recall: 0.9483
2019-01-17T21:04:04.413721, step: 307, loss: 0.23532328009605408, acc: 0.9141, auc: 0.9665, precision: 0.9811, recall: 0.8387
2019-01-17T21:04:05.149400, step: 308, loss: 0.23637895286083221, acc: 0.8906, auc: 0.9673, precision: 0.8961, recall: 0.92
2019-01-17T21:04:05.894358, step: 309, loss: 0.2540436387062073, acc: 0.9062, auc: 0.9662, precision: 0.8873, recall: 0.94

2019-01-17T21:04:47.286610, step: 368, loss: 0.09831704199314117, acc: 0.9766, auc: 0.9973, precision: 0.9706, recall: 0.9851
2019-01-17T21:04:47.966036, step: 369, loss: 0.16715316474437714, acc: 0.9297, auc: 0.9836, precision: 0.9355, recall: 0.9206
2019-01-17T21:04:48.684728, step: 370, loss: 0.0625847727060318, acc: 0.9766, auc: 0.9978, precision: 0.9859, recall: 0.9722
2019-01-17T21:04:49.385805, step: 371, loss: 0.04316984489560127, acc: 0.9922, auc: 1.0, precision: 1.0, recall: 0.9839
2019-01-17T21:04:50.062564, step: 372, loss: 0.06703036278486252, acc: 0.9531, auc: 0.9983, precision: 0.9851, recall: 0.9296
2019-01-17T21:04:50.742647, step: 373, loss: 0.05461795628070831, acc: 0.9844, auc: 0.9995, precision: 1.0, recall: 0.9692
2019-01-17T21:04:51.411710, step: 374, loss: 0.130939319729805, acc: 0.9688, auc: 0.9842, precision: 0.9821, recall: 0.9483
2019-01-17T21:04:52.143966, step: 375, loss: 0.07114975154399872, acc: 0.9844, auc: 0.9968, precision: 0.9672, recall: 1.0
2019-01